# CPSC 330 hw8

## Instructions
rubric={points:5}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330/blob/master/docs/homework_instructions.md). 

## Dataset
rubric={points:5}

For this assignment, you can choose any reasonable dataset for supervised learning. Here is a list of Kaggle datasets in CSV format: https://www.kaggle.com/datasets?fileType=csv. You can also pick a dataset from your own hobby project or research project. 

If it's not already clear from the dataset, you will need to choose which column you are trying to predict. 

This week, you will build ML models on your dataset. Next week (hw9), you'll write a blog post about your work.

Your task here for these 5 points is to describe where you got the data from. If from Kaggle or another online source, poviding a link is sufficient. If from elsewhere, please elaborate.

## Exercise 1
rubric={points:10}

For your dataset: 

- Describe your goal (e.g. what you are trying to predict).
- Describe a (possibly fictional) scenario where someone in the real world is interested in this task and why it is useful to them. 
- What is the **decision** that ML is going to help make and what are the alternatives?

Max 1 paragraph total.

## Exercise 2

Build a model for your chosen problem. Your work must involve the following elements (the list is broken up between the sub-parts):

- Split your data into train/test

#### 2(a)
rubric={points:10}

- Perform exploratory data analysis, including outlier detection. 

#### 2(b)
rubric={points:5}

- Drop features that would not actually be available during deployment.
- Drop features for other reasons, e.g. they would not be useful for predicting the target.
- Provide a brief justification explaining the features you dropped.

#### 2(c)
rubric={points:10}

- Build a preprocessing pipeline.

#### 2(d)
rubric={points:5}

- Try some models includng, at least:
    1. `DummyRegressor` or `DummyClassifer`, as appropriate
    2. `Ridge` or `LogisticRegression`, as appropriate
    3. `LGBMRegressor` or `LGBMClassifier`, as appropriate 


#### 2(e)
rubric={points:5}

- Hyperparameter tuning.

#### 2(f)
rubric={points:5}

- Look at the sub-scores from the different folds of cross-validation and briefly discuss.


#### 2(g)
rubric={points:5}

- Compute at least one relevant scoring metric aside from the default `.score()` and briefly discussing the result.


#### 2(h)
rubric={points:5}

- Evaluate your model on the test set and briefly discuss your level of confidence in whether this is an accurate assessment of your model's future deployment performance.

## Exercise 3
rubric={points:10}

The following short-answer questions pertain to Lecture 19 on clustering.

1. What's the main difference between unsupervised and supervised learning?
2. When choosing $k$ in $k$-means, why not just choose the $k$ that leads to the smallest inertia (sum of squared distances within clusters)?
3. You decide to use clustering for _outlier detection_; that is, to detect instances that are very atypical compared to all the rest. How might you do this with $k$-means?
4. You decide to use clustering for _outlier detection_; that is, to detect instances that are very atypical compared to all the rest. How might you do this with DBSCAN?
5. For hierarchical clustering, we briefly discussed a few different methods for merging clusters: single linkage, average linkage, etc. Why do we have this added complication here - can't we just minimize distance like we did with $k$-means? 

## Submission to Canvas

**IF YOU ARE WORKING WITH A PARTNER** please form the group before submitting - see instructions [here](https://github.com/UBC-CS/cpsc330/blob/master/docs/homework_instructions.md#partners).

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`.
2. Save your notebook.
3. Convert your notebook to `.html` format using the `convert_notebook()` function below **or** by `File -> Export Notebook As... -> Export Notebook to HTML`.
4. Run the code `submit()` below to go through an interactive submission process to Canvas.
>For this step, you will need a Canvas *Access Token* token. If you haven't already got one, log-in to Canvas, click `Account` (top-left of the screen), then `Settings`, then scroll down until you see the `+ New Access Token` button. Click that button, give your token any name you like and set the expiry date to Dec 31, 2020. Then click `Generate token`. Save this token in a safe place on your computer as you'll need it for all assignments. Treat the token with as much care as you would an important password. 

Note: for those having trouble with the Jupyter widgets and the dropdowns: if you add the argument `no_widgets=True` to your `submit` call, it should let you do a text-based entry of your key and avoid the dropdowns altogether. If this doesn't work, you probably need to upgrade to the latest version of `canvasutils` with `pip install canvasutils -U` from your terminal with your environment activated.


In [ ]:
from canvasutils.submit import submit, convert_notebook

# Note: the canvasutils package should have been installed as part of your environment setup - 
# see https://github.com/UBC-CS/cpsc330/blob/master/docs/setup.md

In [ ]:
# convert_notebook("hw8.ipynb", "html")  # uncomment and run when you want to try convert your notebook to HTML (or you can convert manually from the File menu)

In [ ]:
# submit(course_code=53561, token=False)  # uncomment and run when ready to submit 